# How to remove old linux kernel and related modules

Linux Kernel and related files are stored in the following locations on your Linux distro:

- `/boot` – Stores actual kernel and related file such system man and initrd images.
- `/etc` or `/boot/grub` – Stores grub.conf file (most distro use /boot/grub these days).
- `/lib/modules/KERNEL-VERSION/*` (`/lib/modules/$(uname -r)`)- Linux device drivers (modules)

## 1. How to remove the kernels installed with system package manager before

If the Linux kernel is installed using a binary package system, such as apt-get, we can remove it with the same package manager.

### Step 1, check the existed kernel and modules installed.

The kernels and modules installed with system package manager can be checked with `dpkg --list` command.

In [1]:
dpkg --list | grep -E 'linux-headers|kernel-image|linux-image'

ii  linux-headers-5.19.0-32-generic            5.19.0-32.33~22.04.1                    amd64        Linux kernel headers for version 5.19.0 on 64 bit x86 SMP
ii  linux-headers-5.19.0-41-generic            5.19.0-41.42~22.04.1                    amd64        Linux kernel headers for version 5.19.0 on 64 bit x86 SMP
ii  linux-headers-generic-hwe-22.04            5.19.0.41.42~22.04.14                   amd64        Generic Linux kernel headers
ii  linux-image-5.19.0-32-generic              5.19.0-32.33~22.04.1                    amd64        Signed kernel image generic
ii  linux-image-5.19.0-41-generic              5.19.0-41.42~22.04.1                    amd64        Signed kernel image generic
ii  linux-image-generic-hwe-22.04              5.19.0.41.42~22.04.14                   amd64        Generic Linux kernel image


As is showed in the output above, the oldest kernel version is '5.19.0-32', and that's what we're going to remove. Before that, we need to check out the other packages related with this version.

In [2]:
dpkg --list | grep -F '5.19.0-32'

ii  linux-headers-5.19.0-32-generic            5.19.0-32.33~22.04.1                    amd64        Linux kernel headers for version 5.19.0 on 64 bit x86 SMP
ii  linux-hwe-5.19-headers-5.19.0-32           5.19.0-32.33~22.04.1                    all          Header files related to Linux kernel version 5.19.0
ii  linux-image-5.19.0-32-generic              5.19.0-32.33~22.04.1                    amd64        Signed kernel image generic
ii  linux-modules-5.19.0-32-generic            5.19.0-32.33~22.04.1                    amd64        Linux kernel extra modules for version 5.19.0 on 64 bit x86 SMP
ii  linux-modules-extra-5.19.0-32-generic      5.19.0-32.33~22.04.1                    amd64        Linux kernel extra modules for version 5.19.0 on 64 bit x86 SMP


So, there are 5 packages that we need to remove.

### Step 2, uninstall the packages

In [4]:
sudo apt purge -y linux-headers-5.19.0-32-generic \
                  linux-hwe-5.19-headers-5.19.0-32 \
                  linux-image-5.19.0-32-generic \
                  linux-modules-5.19.0-32-generic \
                  linux-modules-extra-5.19.0-32-generic

Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree... Done
Reading state information... 0% Reading state information... 0%Reading state information... Done
The following packages will be REMOVED:
  linux-headers-5.19.0-32-generic* linux-hwe-5.19-headers-5.19.0-32*
  linux-image-5.19.0-32-generic* linux-modules-5.19.0-32-generic*
  linux-modules-extra-5.19.0-32-generic*
0 upgraded, 0 newly installed, 5 to remove and 12 not upgraded.
After this operation, 738 MB disk space will be freed.

(Reading database ... 197922 files and directories currently installed.)
Removing linux-headers-5.19.0-32-generic (5.19.0-32.33~22.04.1) ...
7Progress: [  0%] [..........................................................] 87Progress: [  5%] [##........................................................] 87Progress: [ 10%] [#

### Step 3, check uninstallation result

In [9]:
dpkg --list | grep -F '5.19.0-32'

: 1

In [15]:
ls -1 /lib/modules/*5.19.0-32

ls: cannot access '/lib/modules/*5.19.0-32': No such file or directory


: 2

In [17]:
ls -1 /boot/*5.19.0-32

ls: cannot access '/boot/*5.19.0-32': No such file or directory


: 2

In [14]:
grep '5.19.0-32' /boot/grub/grub.cfg

: 1

The command results shows that the `5.19.0-32` version kernel is purged.

## 2. How to remove manually built and installed kernel

We can't use package manager to remove the kernels and modules that were manually built and installed. So, we need to remove the file from `/boot/*<kernel version>`, `/lib/modules/<kernel version>`, and update grub configuration.

On the following steps, we are going to remove kernel '6.2.0' that's build by myself before.

### Sep 1, remove kernel modules

In ubuntu 22.04, the `.ko` modules are installed in /lib/modules/<your_kernel>. For version 6.2.0, it would be `/lib/modules/6.2.0`.

In [19]:
ls -1A /lib/modules

5.19.0-41-generic
6.2.0


In [23]:
sudo rm -rf /lib/modules/6.2.0

### Step 2, remove the kernel image

In ubuntu 22.04, the images reside within `/boot` directory.

In [21]:
ls -lA /boot/

total 633328
-rw-r--r-- 1 root root    270012 Apr 18 23:38 config-5.19.0-41-generic
-rw-r--r-- 1 root root    275583 May 11 00:32 config-6.2.0
drwx------ 3 root root      4096 Jan  1  1970 efi
drwxr-xr-x 1 root root       126 May 11 19:47 grub
lrwxrwxrwx 1 root root        16 May 11 00:32 initrd.img -> initrd.img-6.2.0
-rw-r--r-- 1 root root  73636557 Apr 29 14:56 initrd.img-5.19.0-41-generic
-rw-r--r-- 1 root root 535160589 May 11 00:36 initrd.img-6.2.0
lrwxrwxrwx 1 root root        28 May 11 00:32 initrd.img.old -> initrd.img-5.19.0-41-generic
-rw-r--r-- 1 root root    182800 Feb  7  2022 memtest86+.bin
-rw-r--r-- 1 root root    184476 Feb  7  2022 memtest86+.elf
-rw-r--r-- 1 root root    184980 Feb  7  2022 memtest86+_multiboot.bin
-rw------- 1 root root   6434470 Apr 18 23:38 System.map-5.19.0-41-generic
-rw-r--r-- 1 root root   7795336 May 11 00:32 System.map-6.2.0
lrwxrwxrwx 1 root root        13 May 11 00:32 vmlinuz -> vmlinuz-6.2.0
-rw------- 1 root root  12196552 Apr 18 23:26 

In [24]:
sudo rm -rf /boot/*6.2.0*

### Step 3, update grub configuration.

In [25]:
sudo grub-mkconfig -o /boot/grub/grub.cfg

Sourcing file `/etc/default/grub'
Sourcing file `/etc/default/grub.d/init-select.cfg'
Generating grub configuration file ...
Found linux image: /boot/vmlinuz-5.19.0-41-generic
Found initrd image: /boot/initrd.img-5.19.0-41-generic
Memtest86+ needs a 16-bit boot, that is not available on EFI, exiting
Systems on them will not be added to the GRUB boot configuration.
Check GRUB_DISABLE_OS_PROBER documentation entry.
Adding boot menu entry for UEFI Firmware Settings ...
done


In [1]:
ls -l /boot/

total 90940
-rw-r--r-- 1 root root   270012 Apr 18 23:38 config-5.19.0-41-generic
drwx------ 3 root root     4096 Jan  1  1970 efi
drwxr-xr-x 1 root root      126 May 11 20:22 grub
lrwxrwxrwx 1 root root       16 May 11 00:32 initrd.img -> initrd.img-6.2.0
-rw-r--r-- 1 root root 73636557 Apr 29 14:56 initrd.img-5.19.0-41-generic
lrwxrwxrwx 1 root root       28 May 11 00:32 initrd.img.old -> initrd.img-5.19.0-41-generic
-rw-r--r-- 1 root root   182800 Feb  7  2022 memtest86+.bin
-rw-r--r-- 1 root root   184476 Feb  7  2022 memtest86+.elf
-rw-r--r-- 1 root root   184980 Feb  7  2022 memtest86+_multiboot.bin
-rw------- 1 root root  6434470 Apr 18 23:38 System.map-5.19.0-41-generic
lrwxrwxrwx 1 root root       13 May 11 00:32 vmlinuz -> vmlinuz-6.2.0
-rw------- 1 root root 12196552 Apr 18 23:26 vmlinuz-5.19.0-41-generic
lrwxrwxrwx 1 root root       25 Apr 29 14:55 vmlinuz.old -> vmlinuz-5.19.0-41-generic


There are broken soft link showed above and we are going to fix them.

In [2]:
sudo rm /boot/initrd.img && sudo ln -s /boot/initrd.img-5.19.0-41-generic /boot/initrd.img

In [3]:
sudo rm /boot/vmlinuz && sudo ln -s /boot/vmlinuz-5.19.0-41-generic /boot/vmlinuz

In [5]:
ls -l /boot/

total 90940
-rw-r--r-- 1 root root   270012 Apr 18 23:38 config-5.19.0-41-generic
drwx------ 3 root root     4096 Jan  1  1970 efi
drwxr-xr-x 1 root root      126 May 11 20:22 grub
lrwxrwxrwx 1 root root       34 May 11 20:27 initrd.img -> /boot/initrd.img-5.19.0-41-generic
-rw-r--r-- 1 root root 73636557 Apr 29 14:56 initrd.img-5.19.0-41-generic
lrwxrwxrwx 1 root root       28 May 11 00:32 initrd.img.old -> initrd.img-5.19.0-41-generic
-rw-r--r-- 1 root root   182800 Feb  7  2022 memtest86+.bin
-rw-r--r-- 1 root root   184476 Feb  7  2022 memtest86+.elf
-rw-r--r-- 1 root root   184980 Feb  7  2022 memtest86+_multiboot.bin
-rw------- 1 root root  6434470 Apr 18 23:38 System.map-5.19.0-41-generic
lrwxrwxrwx 1 root root       31 May 11 20:28 vmlinuz -> /boot/vmlinuz-5.19.0-41-generic
-rw------- 1 root root 12196552 Apr 18 23:26 vmlinuz-5.19.0-41-generic
lrwxrwxrwx 1 root root       25 Apr 29 14:55 vmlinuz.old -> vmlinuz-5.19.0-41-generic
